<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Dominguez
- Nombre de alumno 2: Arturo Lazcano


### **Link de repositorio de GitHub:** `https://github.com/DDominguezD/MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_players = pd.read_csv('/content/drive/MyDrive/stats_players.csv')
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
#Se etiquetan los jugadores. Si el valor en National Postion es nulo, es no seleccionado, si no es el caso, es seleccionado
df_players['etiqueta'] = df_players['National_Position'].apply(lambda x: 'No Seleccionado' if pd.isnull(x) else 'Seleccionado')
df_players['etiqueta'].value_counts()

No Seleccionado    16513
Seleccionado        1075
Name: etiqueta, dtype: int64

In [5]:
# Convierte las etiquetas en valores numéricos
df_players['etiqueta'] = LabelEncoder().fit_transform(df_players['etiqueta'])

# Clase 0 = No seleccionado, Clase 1 = Seleccionado
df_players['etiqueta'].value_counts()

0    16513
1     1075
Name: etiqueta, dtype: int64

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [6]:
#Columnas tipo objeto a preprocesar
column_object = ['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate']

#Columnas numericas a preprocesar
column_int=list(df_players.select_dtypes(['int64']).columns)[:-1]

In [7]:
#Generacion de un ColumnTransformer
preprocessing_transformer = ColumnTransformer(transformers=[
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'), column_object),
    ('scaler', StandardScaler(), column_int)],
    remainder='passthrough')

In [8]:
#Creación de los Pipelines

pipeline_1 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ("svc", SVC(kernel='linear'))
    ])

pipeline_2 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ('knn', KNeighborsClassifier(n_neighbors=3))
    ])

pipeline_3 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ('Random Forest', RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=1))
    ])

Los preprocesamientos a utilizar son `OneHotEncoding()` y `StandardScaler()`. Estos están orientados a las columnas tipo object y numéricas respectivamente.  
Notar que para el preprocesamiento, se dropea la columna con el nombre del jugador y la de national_postition. Esto pues puede estar muy correlacionada a la respuesta que estamos buscando, por lo que el modelo tenderá a ver más esa columna que a las características del jugador en sí.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [9]:
# Separación de los datos en entrenamiento y prueba 
X =  df_players.drop(columns=['Name', 'etiqueta', "National_Position"])
y = df_players.loc[:, "etiqueta"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, stratify=y)

In [10]:
#Entrenamiento de los Pipelines
Pipelines = [pipeline_1, pipeline_2, pipeline_3]
labels = ['SVM lineal', 'KNN', 'Random Forest']

for i, pipeline in enumerate(Pipelines):
  # Entrenamos
  pipeline.fit(X_train, y_train)

  # Predecimos
  y_pred = pipeline.predict(X_test)

  # Evaluamos
  print(f'Métricas para {labels[i]}')
  print(classification_report(y_pred, y_test))
  print(' ')

Métricas para SVM lineal
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      5172
           1       0.24      0.73      0.36       105

    accuracy                           0.95      5277
   macro avg       0.62      0.84      0.67      5277
weighted avg       0.98      0.95      0.96      5277

 
Métricas para KNN
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      5196
           1       0.10      0.38      0.15        81

    accuracy                           0.94      5277
   macro avg       0.54      0.66      0.56      5277
weighted avg       0.98      0.94      0.95      5277

 
Métricas para Random Forest
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5242
           1       0.07      0.69      0.13        35

    accuracy                           0.94      5277
   macro avg       0.54      0.81      0.55      5277

**Respuesta**:


En las celdas anteriores, se probaron 3 métodos de clasificación (junto con el preprocesamiento previo). Estos son SVM con kernel lineal, KNN y Random Forest. Esto pues, según la referencia del diagrama de sklearn, al tener relativamente pocos datos (numéricos), estos son buenos algoritmos para poder realizar experimentos de clasificación.
Luego de haberlos entrenado, se generó un reporte con métricas usuales. Acá se puede observar que, para a clase 0 (no seleccionado) el modelo tiene un rendimiento bastante bueno, sin embargo, esto se debe al desbalance de datos, ya que si ahora se observa la clase 1 (seleccionados) las métricas empeoran en gran medida. Así, para la clase 1, la precisión indica la proporción de instancias clasificadas correctamente como positivas con respecto al total de instancias clasificadas como positivas, mientras que el recall se refiere a la proporción de instancias positivas clasificadas correctamente con respecto al total de instancias positivas.

Como ya se mencionó, esto se debe al desbalance de datos, es por esto que una forma de mejorar estos resutados es aplicar técnicas de balance. Por otro lado, cada modelo de clasificación tiene hiperparámetros que se pueden seleccionar, y una buena búsqueda de ellos también es útil para mejorar el rendimiento.


## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [11]:
def posiciones(x):
  posicion = None
  if x in ['ST', 'CF']:
    posicion = 'ataque'
  elif x in ['RW', 'CAM', 'LW']:
    posicion = 'central_ataque'
  elif x in ['RM', 'CM', 'LM']:
    posicion = 'central'
  elif x in ['RWB', 'CDM', 'LWB']:
    posicion = 'central_defensa'
  elif x in ['RB', 'CB', 'LB']:
    posicion = 'defensa'
  elif x in ['GK']:
    posicion = 'arquero'
  else:
    posicion = x
  return posicion

In [12]:
df_players['etiqueta_club_posicion'] = df_players['Club_Position'].apply(posiciones) #Aplicamos las etiquetas al dataframe

#Filtramos el dataframe solo por estas etiquetas
df_players_position = df_players[df_players['etiqueta_club_posicion'].isin(['ataque', 'central_ataque', 'central', 'central_defensa', 'defensa', 'arquero'])]

#Mostramos la cantidad de valores por clase
df_players_position['etiqueta_club_posicion'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: etiqueta_club_posicion, dtype: int64

In [13]:
#Separamos en train y test
X =  df_players_position.drop(columns=['Name', 'etiqueta', "National_Position", 'etiqueta_club_posicion', 'Club_Position'])
y = df_players_position.loc[:, "etiqueta_club_posicion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, stratify=y)

In [14]:
#Preprocesamiento de los datos

#Columnas objeto a preprocesar
column_object = ['Nationality', 'Preffered_Foot', 'Work_Rate']
#Columnas int a preprocesar
column_int=list(df_players.select_dtypes(['int64']).columns)[:-1]

preprocessing_transformer = ColumnTransformer(transformers=[
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'), column_object),
    ('scaler', StandardScaler(), column_int)],
    remainder='passthrough')

In [15]:
#Creación de los pipelines
pipeline_1 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ("svc", SVC(kernel='linear'))
    ])

pipeline_2 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ('knn', KNeighborsClassifier(n_neighbors=3))
    ])

pipeline_3 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
    ('Random Forest', RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=44))
    ])

In [16]:
#Entrenamiento de los pipelines
Pipelines = [pipeline_1, pipeline_2, pipeline_3]
labels = ['SVM lineal', 'KNN', 'Random Forest']

for i, pipeline in enumerate(Pipelines):
  # Entrenamos
  pipeline.fit(X_train, y_train)

  # Predecimos
  y_pred = pipeline.predict(X_test)

  # Evaluamos
  print(f'Metricas para {labels[i]}')
  print(classification_report(y_pred, y_test))
  print(' ')

Metricas para SVM lineal
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.88      0.79      0.83       143
        central       0.65      0.61      0.63       293
 central_ataque       0.44      0.53      0.48       144
central_defensa       0.19      0.46      0.27        26
        defensa       0.93      0.85      0.89       386

       accuracy                           0.76      1182
      macro avg       0.68      0.71      0.68      1182
   weighted avg       0.79      0.76      0.77      1182

 
Metricas para KNN
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.82      0.62      0.71       170
        central       0.58      0.50      0.54       317
 central_ataque       0.32      0.51      0.39       107
central_defensa       0.08      0.24      0.12        21
        defensa       0.87      0.82   

**Respuesta**:

Al igual que en secciones anteriores, los modelos a utilizar fueron los clasificadores SVM por kernel lineal, KNN y Random Forest.
En estos resultados se puede apreciar como los grupos de "arquero", "ataque" y "defensa" son las clases con mejores métricas, tanto para la precision como para el recall. Por otro lado, la clase "central defensa" parece ser la clase más difícil a predecir por el modelo.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [17]:
#Se cargan los datos
df_sueldos = pd.read_csv('/content/drive/MyDrive/salarios.csv')
df_sueldos

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [18]:
#Se anexa el DataFrame de sueldos al de jugadores
df_info = pd.merge(df_sueldos[['Player','Weekly Salary']], df_players, left_on='Player', right_on='Name')
df_info.drop(['Player', 'etiqueta', 'etiqueta_club_posicion'], axis=1, inplace=True)

In [19]:
# Se separa los datos en train y test
X = df_info.drop(columns=['Name', 'Weekly Salary'])
y = df_info.loc[:, "Weekly Salary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)

In [20]:
#Columnas a Preprocesar
column_object = ['National_Position','Club_Position','Nationality', 'Preffered_Foot', 'Work_Rate']
column_int=list(df_info.select_dtypes(['int64']).columns) 

#Preprocesamiento de los datos
preprocessing_transformer = ColumnTransformer(transformers=[
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'), column_object),
    ('scaler', StandardScaler(), column_int)],
    remainder='passthrough')

#Creación del Pipeline
pipeline_1 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
     ('regresor', LinearRegression())
    ])

pipeline_2 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
     ('regresor', Lasso(alpha=400))
    ])

pipeline_3 = Pipeline([
    ("preprocesamiento", preprocessing_transformer), 
     ('regresor', GradientBoostingRegressor())])

In [21]:
#Entrenamiento de los pipelines
Pipelines = [pipeline_1, pipeline_2, pipeline_3]
labels = ['Lineal', 'Lasso', 'GradientBoosting']

for i, pipeline in enumerate(Pipelines):
  # Entrenamos
  pipeline.fit(X_train, y_train)

  # Predecimos
  y_pred = pipeline.predict(X_test)

  # Evaluamos
  print(f'Metrica para {labels[i]}')
  print('R²:', r2_score(y_test, y_pred))
  print(' ')

Metrica para Lineal
R²: 0.20517310565734226
 
Metrica para Lasso
R²: 0.29418673457462596
 
Metrica para GradientBoosting
R²: 0.6424717881751174
 


**Respuesta**:

El coeficiente de determinación $R^2$ es una métrica utilizada en problemas de regresión para evaluar qué tan bien un modelo se ajusta a los datos observados. Va de $0$ a $1$, donde $1$ indica un ajuste perfecto y $0$ indica un mal ajuste. Un valor más alto de $R^2$ indica que el modelo puede explicar una mayor proporción de la variabilidad de la variable dependiente. Sin embargo, $R^2$ no indica la precisión de las predicciones del modelo en datos no vistos.

En las celdas anteriores se puede ver como se implementan diferentes modelos de regresión (ya no son clasificadores) dados por la guía de sklearn. Esto con el objetivo de predecir el salario que poseerá un jugador.
Primero, se puede notar que la base de datos pasa a ser muy acotada realizando el merge entre stats_player.csv y salarios.csv, mientras que por el lado de las métricas ($R^2$ en este caso), el rendimiento es bastante pobre. Esto ya que superan ligeramente el límite inferior entregado por Renacin.

Es por esto que se decidió probar un modelo de regresión más sofisticado, como GradientBoosting, pues este utiliza árboles de decisión como base y aprende de los errores del modelo anterior para mejorar las predicciones en cada iteración. De esta forma, usando este modelo se consiguió obtener un $R^2$ alrededor de $0.6$.

Posibles tareas para mejorar este rendimiento es realizar otros tipos de preprocesamiento e incluirlos como nuevas características para los modelos junto con una mayor recolección de datos. Por último, buscar hiperparámetros óptimos también puede ayudar a mejorar esta problemática.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>